In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [2]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        term=d*8^o2   
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
od3=[]
m3s=[]
for a in [1..100]: #default precision must be > max a.
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    o3=order(3,c)
    od3=od3+[o3]
    m3=mod(c,3)
    m3s=m3s+[m3]
    ds=digitsum(2,a)
    test=o2-3*ds
    tests=tests+[test]
    print((a,ds,o2,o3,test,m3))
print(tests);print();print(od3)
print();print(m3s)

(1, 1, 3, 0, 0, 1)
(2, 1, 3, 0, 0, 1)
(3, 2, 6, 0, 0, 1)
(4, 1, 3, 0, 0, 1)
(5, 2, 6, 0, 0, 1)
(6, 2, 6, 0, 0, 1)
(7, 3, 9, 0, 0, 1)
(8, 1, 3, 0, 0, 1)
(9, 2, 6, 0, 0, 1)
(10, 2, 6, 0, 0, 1)
(11, 3, 9, 0, 0, 1)
(12, 2, 6, 0, 0, 1)
(13, 3, 9, 0, 0, 1)
(14, 3, 9, 0, 0, 1)
(15, 4, 12, 0, 0, 1)
(16, 1, 3, 0, 0, 1)
(17, 2, 6, 0, 0, 1)
(18, 2, 6, 0, 0, 1)
(19, 3, 9, 0, 0, 1)
(20, 2, 6, 0, 0, 1)
(21, 3, 9, 0, 0, 1)
(22, 3, 9, 0, 0, 1)
(23, 4, 12, 0, 0, 1)
(24, 2, 6, 0, 0, 1)
(25, 3, 9, 0, 0, 1)
(26, 3, 9, 0, 0, 1)
(27, 4, 12, 0, 0, 1)
(28, 3, 9, 0, 0, 1)
(29, 4, 12, 0, 0, 1)
(30, 4, 12, 0, 0, 1)
(31, 5, 15, 0, 0, 1)
(32, 1, 3, 0, 0, 1)
(33, 2, 6, 0, 0, 1)
(34, 2, 6, 0, 0, 1)
(35, 3, 9, 0, 0, 1)
(36, 2, 6, 0, 0, 1)
(37, 3, 9, 0, 0, 1)
(38, 3, 9, 0, 0, 1)
(39, 4, 12, 0, 0, 1)
(40, 2, 6, 0, 0, 1)
(41, 3, 9, 0, 0, 1)
(42, 3, 9, 0, 0, 1)
(43, 4, 12, 0, 0, 1)
(44, 3, 9, 0, 0, 1)
(45, 4, 12, 0, 0, 1)
(46, 4, 12, 0, 0, 1)
(47, 5, 15, 0, 0, 1)
(48, 2, 6, 0, 0, 1)
(49, 3, 9, 0, 0, 1)
(50, 3, 9, 0, 0, 1

In [3]:
import random
import time


def specialModulus(a):
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n):
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    A_n=lcm(mlist) 
    ans=0
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        term=d*8^o2+r*A_n 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    
bound=500
L=LaurentSeriesRing(QQ,'x', default_prec=bound+10)

tests=[]
for a in [1..bound]: #default precision must be > max a.
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    ds=digitsum(2,a)
    test=o2-3*ds
    tests=tests+[test]
    print((a,ds,o2))
print(tests)

(1, 1, 3)
(2, 1, 3)
(3, 2, 6)
(4, 1, 3)
(5, 2, 6)
(6, 2, 6)
(7, 3, 9)
(8, 1, 3)
(9, 2, 6)
(10, 2, 6)
(11, 3, 9)
(12, 2, 6)
(13, 3, 9)
(14, 3, 9)
(15, 4, 12)
(16, 1, 3)
(17, 2, 6)
(18, 2, 6)
(19, 3, 9)
(20, 2, 6)
(21, 3, 9)
(22, 3, 9)
(23, 4, 12)
(24, 2, 6)
(25, 3, 9)
(26, 3, 9)
(27, 4, 12)
(28, 3, 9)
(29, 4, 12)
(30, 4, 12)
(31, 5, 15)
(32, 1, 3)
(33, 2, 6)
(34, 2, 6)
(35, 3, 9)
(36, 2, 6)
(37, 3, 9)
(38, 3, 9)
(39, 4, 12)
(40, 2, 6)
(41, 3, 9)
(42, 3, 9)
(43, 4, 12)
(44, 3, 9)
(45, 4, 12)
(46, 4, 12)
(47, 5, 15)
(48, 2, 6)
(49, 3, 9)
(50, 3, 9)
(51, 4, 12)
(52, 3, 9)
(53, 4, 12)
(54, 4, 12)
(55, 5, 15)
(56, 3, 9)
(57, 4, 12)
(58, 4, 12)
(59, 5, 15)
(60, 4, 12)
(61, 5, 15)
(62, 5, 15)
(63, 6, 18)
(64, 1, 3)
(65, 2, 6)
(66, 2, 6)
(67, 3, 9)
(68, 2, 6)
(69, 3, 9)
(70, 3, 9)
(71, 4, 12)
(72, 2, 6)
(73, 3, 9)
(74, 3, 9)
(75, 4, 12)
(76, 3, 9)
(77, 4, 12)
(78, 4, 12)
(79, 5, 15)
(80, 2, 6)
(81, 3, 9)
(82, 3, 9)
(83, 4, 12)
(84, 3, 9)
(85, 4, 12)
(86, 4, 12)
(87, 5, 15)
(88, 3, 9)
(89, 4, 12